# 04 - Abstractions: Building Flexible Systems

**Learning Goal**: Master abstraction patterns that enable swappable implementations and extensible architecture.

---

## 🎯 Problem: Why Abstractions?

### The Multi-Engine Challenge

Imagine building a data processing library that needs to support:
- **Pandas** for small/medium datasets on a single machine
- **Spark** for large-scale distributed processing
- **Polars** for high-performance single-machine workloads

**Without abstractions:**
```python
def read_data(engine_type: str, path: str):
    if engine_type == 'pandas':
        return pd.read_csv(path)
    elif engine_type == 'spark':
        return spark.read.csv(path)
    elif engine_type == 'polars':
        return pl.read_csv(path)
    # What happens when we add DuckDB? Dask? Modin?
```

Problems:
- ❌ If/else chains everywhere
- ❌ Tight coupling to specific implementations
- ❌ Can't add new engines without modifying existing code
- ❌ No compile-time guarantee that all engines implement all operations

**With abstractions:**
```python
class Engine(ABC):
    @abstractmethod
    def read(self, path: str) -> DataFrame: ...

def read_data(engine: Engine, path: str):
    return engine.read(path)  # Works with ANY engine!
```

Benefits:
- ✅ Single interface, multiple implementations
- ✅ Add new engines without changing existing code
- ✅ Type-safe contracts enforced at definition time
- ✅ Easy to test with mock implementations

## 🦉 First Principles: Depend on Abstractions, Not Concretions

### The Dependency Inversion Principle

**High-level modules should not depend on low-level modules. Both should depend on abstractions.**

```python
# ❌ BAD: Depends on concretion
class DataPipeline:
    def __init__(self):
        self.engine = PandasEngine()  # Locked to Pandas!

# ✅ GOOD: Depends on abstraction
class DataPipeline:
    def __init__(self, engine: Engine):  # Works with any Engine!
        self.engine = engine
```

### Python's Abstraction Tools

1. **ABC (Abstract Base Classes)**: Explicit contracts with inheritance
2. **Protocol**: Structural subtyping (duck typing with type safety)
3. **Duck Typing**: "If it walks like a duck..."

### When to Use Each

| Pattern | Use When | Example |
|---------|----------|----------|
| **ABC** | You control implementations & want enforced contracts | Odibi's Engine (internal library) |
| **Protocol** | Working with external types or want structural typing | File-like objects, Iterables |
| **Duck Typing** | Maximum flexibility, runtime checking acceptable | Quick scripts, exploratory code |

## ⚡ Examples: ABC vs Protocol

### Example 1: ABC for Explicit Contracts

In [ ]:
from abc import ABC, abstractmethod
from typing import Any

class DataReader(ABC):
    """Abstract interface for data readers."""
    
    @abstractmethod
    def read(self, source: str) -> Any:
        """Read data from source."""
        pass
    
    @abstractmethod
    def get_schema(self) -> dict:
        """Return schema information."""
        pass

# ❌ This will fail instantly - can't instantiate abstract class
# reader = DataReader()  # TypeError!

class CSVReader(DataReader):
    def read(self, source: str) -> list[dict]:
        # Actual implementation
        import csv
        with open(source) as f:
            return list(csv.DictReader(f))
    
    def get_schema(self) -> dict:
        return {"type": "csv", "columns": []}

# ✅ This works - all abstract methods implemented
reader = CSVReader()
print(f"Created: {type(reader).__name__}")
print(f"Is DataReader? {isinstance(reader, DataReader)}")

### Example 2: Protocol for Structural Subtyping

In [ ]:
from typing import Protocol, runtime_checkable

@runtime_checkable
class Readable(Protocol):
    """Protocol: anything with a read() method."""
    def read(self, source: str) -> Any: ...

# No inheritance needed!
class JSONReader:
    def read(self, source: str) -> dict:
        import json
        with open(source) as f:
            return json.load(f)

class ParquetReader:
    def read(self, source: str) -> Any:
        # Pretend implementation
        return {"data": "from parquet"}

# ✅ Both satisfy the protocol automatically
def process_data(reader: Readable, source: str):
    return reader.read(source)

json_reader = JSONReader()
parquet_reader = ParquetReader()

print(f"JSONReader is Readable? {isinstance(json_reader, Readable)}")
print(f"ParquetReader is Readable? {isinstance(parquet_reader, Readable)}")

# Type checker accepts both even though they don't inherit from Readable!

### Example 3: Composition Over Inheritance

In [ ]:
from dataclasses import dataclass

# ❌ Inheritance approach - rigid hierarchy
class DatabaseReader(DataReader):
    def connect(self): ...
    def read(self, query: str): ...
    def get_schema(self): ...

class CachedDatabaseReader(DatabaseReader):
    def read(self, query: str):  # Must override
        # Check cache, then call super().read()
        pass

# ✅ Composition approach - flexible and testable
@dataclass
class CachedReader:
    """Wraps any reader with caching."""
    reader: DataReader
    cache: dict[str, Any] = None
    
    def __post_init__(self):
        if self.cache is None:
            self.cache = {}
    
    def read(self, source: str) -> Any:
        if source not in self.cache:
            self.cache[source] = self.reader.read(source)
        return self.cache[source]
    
    def get_schema(self) -> dict:
        return self.reader.get_schema()

# Works with ANY DataReader implementation
csv_reader = CSVReader()
cached_csv = CachedReader(csv_reader)

print("Composition allows mixing behaviors freely!")
print(f"Base reader: {type(cached_csv.reader).__name__}")

## 🔍 Odibi Analysis: Production Abstraction Patterns

Let's analyze how Odibi uses ABC to enable multi-engine support.

### 1. Engine Abstraction

**File**: `odibi/engine/base.py`

**Pattern**: ABC with 9 abstract methods defining complete engine contract

```python
class Engine(ABC):
    @abstractmethod
    def read(self, connection, format, table=None, path=None, options=None) -> Any: ...
    
    @abstractmethod
    def write(self, df, connection, format, table=None, path=None, mode="overwrite", options=None) -> None: ...
    
    @abstractmethod
    def execute_sql(self, sql: str, context: Context) -> Any: ...
    
    # ... 6 more abstract methods
```

**Why ABC instead of Protocol?**
- ✅ Odibi controls both the interface AND implementations (PandasEngine, SparkEngine)
- ✅ Wants compile-time errors if implementation forgets a method
- ✅ Clear documentation of the complete contract
- ✅ Prevents accidental instantiation of incomplete engines

**Key Design Decisions**:
1. **Return `Any` instead of concrete types**: Pandas returns `pd.DataFrame`, Spark returns `pyspark.sql.DataFrame`
2. **Consistent method signatures**: All engines implement same interface
3. **Context passing**: Enables cross-engine state management

### 2. Connection Abstraction

**File**: `odibi/connections/base.py`

```python
class BaseConnection(ABC):
    @abstractmethod
    def get_path(self, relative_path: str) -> str:
        """Resolve relative path to full path."""
        pass
    
    @abstractmethod
    def validate(self) -> None:
        """Validate connection config."""
        pass
```

**Why such a minimal interface?**
- Different connection types have vastly different needs:
  - `FileConnection`: Just needs a base directory
  - `DatabaseConnection`: Needs host, port, credentials, connection pool
  - `CloudConnection`: Needs auth tokens, regions, buckets
- Only abstracts the **common operations** all connections must provide
- Specific connection types extend with their own methods

### 3. Context Abstraction

**File**: `odibi/context.py`

```python
class Context(ABC):
    @abstractmethod
    def register(self, name: str, df: Any) -> None: ...
    
    @abstractmethod
    def get(self, name: str) -> Any: ...
    
    @abstractmethod
    def has(self, name: str) -> bool: ...
    
    @abstractmethod
    def list_names(self) -> list[str]: ...
    
    @abstractmethod
    def clear(self) -> None: ...
```

**Implementation Difference**:
- `PandasContext`: Stores DataFrames in dictionary (`self._data`)
- `SparkContext`: Registers temp views in Spark catalog (`createOrReplaceTempView`)

**Why this matters**: SQL queries need to reference DataFrames by name. Each engine handles this differently:
```python
# Pandas: DuckDB queries an in-memory dict
duckdb.query("SELECT * FROM sales", context._data)

# Spark: Queries use temp views in catalog
spark.sql("SELECT * FROM sales")  # Looks up temp view
```

Same interface, completely different storage mechanisms!

## 🏗️ Build: Abstract Data Reader

Let's build a realistic abstraction layer for data reading.

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Any, Protocol
from pathlib import Path

@dataclass
class DataSource:
    """Configuration for a data source."""
    path: str
    format: str
    options: dict[str, Any] | None = None

class AbstractReader(ABC):
    """Base class for all data readers."""
    
    @abstractmethod
    def read(self, source: DataSource) -> Any:
        """Read data from source."""
        pass
    
    @abstractmethod
    def supports_format(self, format: str) -> bool:
        """Check if this reader supports the given format."""
        pass
    
    @abstractmethod
    def get_metadata(self, source: DataSource) -> dict:
        """Get metadata without reading full data."""
        pass

In [ ]:
# Implementation 1: Pandas Reader
import pandas as pd

class PandasReader(AbstractReader):
    """Pandas-based data reader."""
    
    SUPPORTED_FORMATS = {"csv", "parquet", "json", "excel"}
    
    def read(self, source: DataSource) -> pd.DataFrame:
        options = source.options or {}
        
        if source.format == "csv":
            return pd.read_csv(source.path, **options)
        elif source.format == "parquet":
            return pd.read_parquet(source.path, **options)
        elif source.format == "json":
            return pd.read_json(source.path, **options)
        elif source.format == "excel":
            return pd.read_excel(source.path, **options)
        else:
            raise ValueError(f"Unsupported format: {source.format}")
    
    def supports_format(self, format: str) -> bool:
        return format in self.SUPPORTED_FORMATS
    
    def get_metadata(self, source: DataSource) -> dict:
        # Read first row to get schema
        df = self.read(source)
        return {
            "columns": list(df.columns),
            "dtypes": {col: str(dtype) for col, dtype in df.dtypes.items()},
            "row_count": len(df),
            "file_size": Path(source.path).stat().st_size if Path(source.path).exists() else 0
        }

# Test it
reader = PandasReader()
print(f"Supports CSV? {reader.supports_format('csv')}")
print(f"Supports XML? {reader.supports_format('xml')}")

In [ ]:
# Implementation 2: Simulated Spark Reader

class SparkReader(AbstractReader):
    """Spark-based data reader (simulated)."""
    
    SUPPORTED_FORMATS = {"csv", "parquet", "json", "delta", "orc"}
    
    def __init__(self, spark_session=None):
        self.spark = spark_session  # Would be real SparkSession
    
    def read(self, source: DataSource) -> dict:  # Simulating Spark DataFrame
        options = source.options or {}
        
        # In real implementation: self.spark.read.format(source.format).options(**options).load(source.path)
        return {
            "type": "spark_dataframe",
            "format": source.format,
            "path": source.path,
            "options": options
        }
    
    def supports_format(self, format: str) -> bool:
        return format in self.SUPPORTED_FORMATS
    
    def get_metadata(self, source: DataSource) -> dict:
        # In real implementation: df.schema, df.count(), etc.
        return {
            "format": source.format,
            "path": source.path,
            "distributed": True
        }

# Test it
spark_reader = SparkReader()
print(f"Supports Delta? {spark_reader.supports_format('delta')}")
print(f"Supports Excel? {spark_reader.supports_format('excel')}")

In [ ]:
# Factory Pattern: Choose reader based on scale

class ReaderFactory:
    """Factory for creating appropriate readers."""
    
    @staticmethod
    def create_reader(engine: str) -> AbstractReader:
        if engine == "pandas":
            return PandasReader()
        elif engine == "spark":
            return SparkReader()
        else:
            raise ValueError(f"Unknown engine: {engine}")
    
    @staticmethod
    def auto_select(file_size_mb: float) -> AbstractReader:
        """Automatically select reader based on data size."""
        if file_size_mb < 1000:  # Less than 1GB
            return PandasReader()
        else:
            return SparkReader()

# Usage: Client code doesn't care about implementation
def load_data(source: DataSource, engine: str):
    reader = ReaderFactory.create_reader(engine)
    
    if not reader.supports_format(source.format):
        raise ValueError(f"{engine} doesn't support {source.format}")
    
    return reader.read(source)

# Test with both engines
source = DataSource(path="data.csv", format="csv")

pandas_data = load_data(source, engine="pandas")
print(f"Loaded with Pandas: {type(pandas_data)}")

spark_data = load_data(source, engine="spark")
print(f"Loaded with Spark: {spark_data['type']}")

## ✅ Testing Abstract Implementations

Abstractions make testing incredibly easy.

In [ ]:
# Mock implementation for testing
class MockReader(AbstractReader):
    """Test double for data readers."""
    
    def __init__(self, mock_data: Any):
        self.mock_data = mock_data
        self.read_count = 0
    
    def read(self, source: DataSource) -> Any:
        self.read_count += 1
        return self.mock_data
    
    def supports_format(self, format: str) -> bool:
        return True  # Mock supports everything
    
    def get_metadata(self, source: DataSource) -> dict:
        return {"mock": True}

# Use mock in tests
def process_pipeline(reader: AbstractReader, source: DataSource):
    """Example pipeline that works with any reader."""
    metadata = reader.get_metadata(source)
    data = reader.read(source)
    return {"metadata": metadata, "data": data}

# Test without hitting real files
mock_reader = MockReader(mock_data={"test": "data"})
result = process_pipeline(mock_reader, DataSource("fake.csv", "csv"))

print(f"Read called {mock_reader.read_count} times")
print(f"Result: {result}")

## 🎯 Key Takeaways

### When to Use ABC
- ✅ You control the implementations
- ✅ Want compile-time contract enforcement
- ✅ Clear inheritance hierarchy
- ✅ Need to prevent incomplete implementations

### When to Use Protocol
- ✅ Working with external types
- ✅ Want structural subtyping
- ✅ Retrofitting interfaces to existing code
- ✅ Type checking without inheritance

### Design Principles
1. **Depend on abstractions, not concretions**
2. **Keep interfaces minimal** (fewer methods = easier to implement)
3. **Use composition over inheritance** (more flexible)
4. **Factory patterns** for creating implementations
5. **Mock implementations** for testing

### Odibi's Architecture
- **Engine ABC**: Enforces complete contract for Pandas/Spark
- **Connection ABC**: Minimal interface for path resolution
- **Context ABC**: Different storage strategies per engine
- **Factory functions**: `create_context(engine)` selects implementation

**The Pattern**: Define once, implement many times, use everywhere.

## 🚀 Next Steps

1. **exercises.ipynb**: Build storage abstraction, validator abstraction
2. **odibi_abstractions.md**: Deep dive into Odibi's full engine implementations
3. **Advanced topics**: Multiple inheritance, MRO, mixin patterns